# NLP with Bert for Sentiment Analysis

### Importing the libraries

In [1]:
!pip3 install ktrain

     |████████████████████████████████| 25.2MB 128kB/s 
     |████████████████████████████████| 421.8MB 24kB/s 
     |████████████████████████████████| 6.7MB 40.4MB/s 
     |████████████████████████████████| 983kB 12.9MB/s 
     |████████████████████████████████| 245kB 34.3MB/s 
     |████████████████████████████████| 778kB 29.3MB/s 
     |████████████████████████████████| 471kB 31.9MB/s 
     |████████████████████████████████| 3.9MB 32.6MB/s 
     |████████████████████████████████| 450kB 32.0MB/s 
     |████████████████████████████████| 890kB 32.0MB/s 
     |████████████████████████████████| 1.1MB 36.5MB/s 
     |████████████████████████████████| 3.0MB 32.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.18.4-cp36-none-any.whl size=25253172 sha256=8a573842bfb1862c8d5e533d6176540cfc61d4969eb8f790a8006e8c8c251280
  Stored in directory: /root/.cache/pip/wheels/73/ca/65/2bff349d7d2f2e0710d993ddb5d9a2ff2431c617ed3a7a68d7
  Created wheel for keras-bert: filename=keras_bert-0.85.0-cp36-no

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import ktrain
from ktrain import text

## Part 1: Data Preprocessing

In [3]:
train = pd.read_csv('train_E6oV3lV.csv')
test = pd.read_csv('test_tweets_anuFYb8.csv')

In [4]:
train.shape

(31962, 3)

In [5]:
test.shape

(17197, 2)

In [6]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [7]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


### Creating the training and test sets

In [8]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(train,
                                                                   text_column='tweet',
                                                                   label_columns=['label'],
                                                                   maxlen=500,
                                                                   preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


## Part 2: Building the BERT model

In [9]:
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


## Part 3: Training the BERT model

In [10]:
learner = ktrain.get_learner(model=model,
                             train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

In [11]:
learner.fit_onecycle(lr=2e-5,
                     epochs=1)



begin training using onecycle policy with max lr of 2e-05...
Train on 28765 samples, validate on 3197 samples
28765/28765 [==============================] - 8350s 290ms/sample - loss: 0.1309 - accuracy: 0.9549 - val_loss: 0.1007 - val_accuracy: 0.9659


In [12]:
learner.validate(val_data=(x_test, y_test))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2964
           1       0.87      0.63      0.73       233

    accuracy                           0.97      3197
   macro avg       0.92      0.81      0.85      3197
weighted avg       0.96      0.97      0.96      3197



array([[2942,   22],
       [  87,  146]])

## How to Use Our Trained BERT Model

We can call the `learner.get_predictor` method to obtain a Predictor object capable of making predictions on new raw data.

In [13]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [14]:
predictor.get_classes()

['0', '1']

In [15]:
test_tweets = test['tweet'].values
test_tweets

array(['#studiolife #aislife #requires #passion #dedication #willpower   to find #newmaterialsâ\x80¦ ',
       ' @user #white #supremacists want everyone to see the new â\x80\x98  #birdsâ\x80\x99 #movie â\x80\x94 and hereâ\x80\x99s why  ',
       'safe ways to heal your #acne!!    #altwaystoheal #healthy   #healing!! ',
       ...,
       '#hillary #campaigned today in #ohio((omg)) &amp; used words like "assets&amp;liability" never once did #clinton say thee(word) #radicalization   ',
       'happy, at work conference: right mindset leads to culture-of-development organizations    #work #mindset',
       'my   song "so glad" free download!  #shoegaze #newmusic #newsong'],
      dtype=object)

In [16]:
test_tweets[0]

'#studiolife #aislife #requires #passion #dedication #willpower   to find #newmaterialsâ\x80¦ '

In [17]:
predictor.predict(test_tweets[0])

'0'

In [18]:
test_tweets[1]

' @user #white #supremacists want everyone to see the new â\x80\x98  #birdsâ\x80\x99 #movie â\x80\x94 and hereâ\x80\x99s why  '

In [19]:
predictor.predict(test_tweets[1])

'1'

In [20]:
test_tweets[179]

'seeing war craft in imax 3d ð\x9f\x98\x84 #warcraftmovie   #imax #woohoo '

In [21]:
predictor.predict(test_tweets[179])

'0'

In [22]:
test_pred = predictor.predict(test_tweets)

In [23]:
test_pred

['0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0'

In [24]:
test['label'] = test_pred

submission = test[['id','label']]

submission.to_csv('test_predictions.csv', index=False)

In [25]:
res = pd.read_csv('test_predictions.csv')

In [26]:
res

,id,label
0,31963,0
1,31964,1
2,31965,0
3,31966,0
4,31967,0
...,...,...
17192,49155,1
17193,49156,0
17194,49157,1
17195,49158,0


The `predictor.save` and `ktrain.load_predictor` methods can be used to save the Predictor object to disk and reload it at a later time to make predictions on new data.